In [1]:
pip install scikit-learn xgboost joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from joblib import dump, load
import pandas as pd
import numpy as np


In [3]:
# Load the dataset you created
df = pd.read_csv("descriptor_based_dataset.csv")
df = df.copy()
df = df[df['Kd'].notnull()]  # Remove NaN
df = df[np.isfinite(df['Kd'])]  # Remove inf/-inf

X = df.drop("Kd", axis=1)
y = df["Kd"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)


In [4]:
df.describe()

,MolWt,LogP,RotatableBonds,HDonors,HAcceptors,Prot_MW,Aromaticity,Instability,Hydropathy,Kd
count,61092.000000,61092.000000,61092.000000,61092.000000,61092.000000,61092.000000,61092.000000,61092.000000,61092.000000,61092.000000
mean,493.110912,3.365010,7.312643,2.777319,6.906813,75943.994115,0.084169,46.450565,-0.356622,6.055919
std,296.495108,3.025195,8.498220,3.995878,5.662067,54539.680930,0.020120,10.518773,0.297445,1.576809
min,46.073000,-48.012300,0.000000,0.000000,0.000000,2448.512800,0.000000,0.307930,-1.610233,0.020000
25%,370.384000,2.544500,4.000000,1.000000,5.000000,44460.111700,0.070175,40.827940,-0.522418,5.000000
50%,450.345000,3.633300,6.000000,2.000000,6.000000,59353.787900,0.082873,46.330469,-0.370892,5.451737
75%,527.669000,5.024200,8.000000,3.000000,8.000000,92849.581300,0.097222,52.685901,-0.238547,7.050610
max,10112.436000,16.784400,178.000000,98.000000,276.000000,794048.925800,0.186441,90.878018,1.171429,15.221849


In [5]:
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42)),
    ('gbr', GradientBoostingRegressor(n_estimators=100, random_state=42))
]

meta_model = LinearRegression()

stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=5)

In [6]:
stacking_model.fit(X_train, y_train)


StackingRegressor(cv=5,
                  estimators=[('rf', RandomForestRegressor(random_state=42)),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_p...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                              ('gbr',
                               GradientBoostingRegressor(random_state=42))],
                  final_estimator=LinearRegression())

In [7]:
y_pred = stacking_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print("Stacked Ensemble MSE:", mse)

Stacked Ensemble MSE: 1.0872787307383882


In [8]:
dump(stacking_model, 'stacked_model.joblib')

['stacked_model.joblib']